In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [4]:
design.get_chip_z('Q_chip')

'7um'

In [30]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='450um',
                                            pad_height='130um',
                                            pad_gap='100um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+110}um",
                                              pad_width="200um",
                                              pad_height="140um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [34]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '904um', '100um')
            

# EPR Analysis

In [35]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [36]:
hfss.start()

INFO 04:51PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:51PM [load_ansys_project]: 	Opened Ansys App
INFO 04:51PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:51PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   V8
INFO 04:51PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 04:51PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:51PM [connect]: 	Connected to project "V8" and design "Design_q3d" 😀 



True

In [37]:
hfss.activate_ansys_design("flip7um_V9", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

04:51PM 14s WARNING [activate_ansys_design]: The design_name=flip7um_V9 was not in active project.  Designs in active project are: 
['flip7um_V8', 'Design_q3d', 'flip7um_V8-'].  A new design will be added to the project.  
INFO 04:51PM [connect_design]: 	Opened active design
	Design:    flip7um_V9 [Solution type: Eigenmode]
WARNING 04:51PM [connect_setup]: 	No design setup detected.
WARNING 04:51PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:51PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [38]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [39]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [40]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.97',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.97',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [42]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:15PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpcn_u_9ym.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 1, False
INFO 05:15PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3cgu0p4u.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 1, False
INFO 05:15PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0mw9tcig.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 2, False
INFO 05:15PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpzfo_zma0.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 3, False
INFO 05:15PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwfahk_z6.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 4, False
INFO 05:15

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,172.71,-16.610,0.000,-155.30
pad_top_Q0,-16.61,172.090,-34.528,-119.72
quad_coupler_c0,0.00,-34.528,152.190,-117.25
SignalNet1,-155.30,-119.720,-117.250,456.88


In [45]:
c1.setup.junctions = Dict({'Lj': 7, 'Cj': 2})
c1.setup.freq_readout = 6.82
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.827213 [GHz]
EC 194.550339 [MHz]
EJ 23.342229 [GHz]
alpha -211.201332 [MHz]
dispersion 0.000414 [KHz]
Lq 6.997177 [nH]
Cq 99.564093 [fF]
T1 31.637478 [us]

**Coupling Properties**

tCqbus1 -7.680111 [fF]
gbus1_in_MHz -92.528848 [MHz]
χ_bus1 -3.048529 [MHz]
1/T1bus1 5030.582475 [Hz]
T1bus1 31.637478 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.539444,247.281027,23.342229,-271.731974,0.012751,[-48.56543704803131],[-8.279917670693413],8.279918,48.565437
2,5.80227,192.826999,23.342229,-209.243541,0.000362,[-54.715682439912754],[-1.0113123247381683],1.011312,54.715682
3,5.788445,191.875409,23.342229,-208.163058,0.000335,[-72.84108831176982],[-1.7414432396969393],1.741443,72.841088
4,5.779111,191.234335,23.342229,-207.435378,0.000319,[-74.52817957380502],[-1.7882232057149927],1.788223,74.528180
5,5.772234,190.762715,23.342229,-206.900161,0.000307,[-82.99069208327808],[-2.1862622650479673],2.186262,82.990692
6,5.764126,190.207536,23.342229,-206.270245,0.000294,[-83.84848149555815],[-2.1950055652560785],2.195006,83.848481
7,5.806796,193.139101,23.342229,-209.598005,0.000371,[-88.04479668460407],[-2.6435630899065137],2.643563,88.044797
8,5.791013,192.05197,23.342229,-208.363504,0.00034,[-89.38204241017493],[-2.6361533960852994],2.636153,89.382042
9,5.786878,191.767679,23.342229,-208.040762,0.000332,[-89.35018088664643],[-2.6117831231515574],2.611783,89.350181
10,5.827213,194.550339,23.342229,-211.201332,0.000414,[-92.52884822246699],[-3.048529318881654],3.048529,92.528848


In [46]:
c1.sim.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down
